# Метод ветвей и границ(Акули Кирилл)

Подключаем библиотеки для нахождеия оптимального решения задач линейного программирования

In [1]:
from scipy.optimize import linprog

Система:

имеет пододящий вид для подачи ее в алгоритм

In [2]:
import math
import networkx as nx

G = nx.DiGraph()

def zeros(length):
    return [0 for _ in range(length)]

def borders_and_branches_algorithm(c,lhs_ineq,rhs_ineq,bnd,opt_old):
    print("Left A:" + str(lhs_ineq))
    print("Right A:"+str(rhs_ineq))
    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,
              method="revised simplex")
    
    #G.add_node( str(opt.x)+" "+ str(opt.func))
    #if opt_old is not None:
    #    G.add_edge(opt_old,str(opt.x)+" "+ str(opt.func))

    print(opt)
    if not opt.success:
        opt.func =  float('inf')
        return opt

    x_values = opt.x
    print("============================" )
    for i,x in enumerate(x_values):
        if x.is_integer():
            continue
        ceil_lhs_ineq = lhs_ineq.copy()
        ceil_lhs_ineq_row = zeros(len(c))
        ceil_lhs_ineq_row[i] = -1
        ceil_lhs_ineq.append(ceil_lhs_ineq_row)
        ceil_rhs_ineq = rhs_ineq.copy()
        ceil_rhs_ineq.append((-1)* math.ceil(x))
        ceil_solution = borders_and_branches_algorithm(c,ceil_lhs_ineq, ceil_rhs_ineq,bnd,"")

        floor_lhs_ineq = lhs_ineq.copy()
        floor_lhs_ineq_row = zeros(len(c))
        floor_lhs_ineq_row[i] = 1
        floor_lhs_ineq.append(floor_lhs_ineq_row)
        floor_rhs_ineq = rhs_ineq.copy()
        floor_rhs_ineq.append(math.floor(x))
        floor_solution = borders_and_branches_algorithm(c,floor_lhs_ineq, floor_rhs_ineq,bnd,"")

        if ceil_solution.success:
            return floor_solution
        if floor_solution.success:
            return ceil_solution
        
        return min(ceil_solution,floor_solution,key= lambda x: x.func)
    
    return opt

    


In [14]:
# function to optimaze params
obj  = [-1,-1]

# constraint matrix
lhs_ineq = [[ 2,  11], 
            [1,  1],  
            [ 4, -5]]

rhs_ineq = [ 38,  # правая сторона красного неравенства
            7,  # правая сторона синего неравенства
            5]

bnd = [(0, float("inf")),  # x1 x2 constraints 
       (0, float("inf"))] 
       
print(borders_and_branches_algorithm(obj,lhs_ineq,rhs_ineq,bnd,None))


Left A:[[2, 11], [1, 1], [4, -5]]
Right A:[38, 7, 5]
     con: array([], dtype=float64)
     fun: -7.0
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([ 1.00000000e+00,  0.00000000e+00, -1.77635684e-15])
  status: 0
 success: True
       x: array([4.44444444, 2.55555556])
Left A:[[2, 11], [1, 1], [4, -5], [-1, 0]]
Right A:[38, 7, 5, -5]
     con: array([], dtype=float64)
     fun: -7.0
 message: 'The problem appears infeasible, as the phase one auxiliary problem terminated successfully with a residual of 5.0e+00, greater than the tolerance 1e-12 required for the solution to be considered feasible. Consider increasing the tolerance to be greater than 5.0e+00. If this tolerance is unnaceptably large, the problem is likely infeasible.'
     nit: 1
   slack: array([ 6.,  0., -5.,  0.])
  status: 2
 success: False
       x: array([5., 2.])
Left A:[[2, 11], [1, 1], [4, -5], [1, 0]]
Right A:[38, 7, 5, 4]
     con: array([], dtype=float64)
     fun: -6.72727272727